In [1]:
import pandas as pd
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize

In [2]:
df = pd.read_parquet("data/item_description.parquet")
df.head(50)

,ITEM_CODE,ITEM_DESC,PRINCIPAL_CODE,PRINCIPAL_DESC,DIRECTORAT_ITEM_CODE,DIRECTORAT_ITEM_DESC,CLASS_PRODUCT_CODE,CLASS_PRODUCT_DESC
0,00351,C-A-MATE XM 220 SYSTEM,702_1111,EMP-HOSLAB (PHARMAMED),1111,PHARMAMED,7024090,EMP-HOSLAB BLOOD ACCESORIES
1,CMYCD,MYCORAL CREAM 5 GR,803_1111,KALBE NIMITZ (PHARMAMED),1111,PHARMAMED,8011105,NIMITZ
2,CMYCF,MYCORAL CREAM 15 GR,803_1111,KALBE NIMITZ (PHARMAMED),1111,PHARMAMED,8011105,NIMITZ
3,01501,"DUODENAL LEVIN, CH 16",701_1111,EMP-HOSPEC (PHARMAMED),1111,PHARMAMED,7010890,EMP - HOSPEC LAIN-LAIN
4,CNZDA,NAZODERM CREAM 10 GR,002_1111,PT.DANKOS FARMA (PHARMAMED),1111,PHARMAMED,0021101,DANKOS GABUNGAN
5,COSKA,OVISKIN CREAM 5 GR,801_1111,HEXPHARM (PHARMAMED),1111,PHARMAMED,8011117,HEXPHARM - AP 3
6,COSKB,OVISKIN CREAM 10 GR,365_1111,HEXPHARM TSJ (PHARMAMED),1111,PHARMAMED,3650001,HEXPHARM TSJ
7,COXZA,OXYZONE 5 GRAM CREAM,801_1111,HEXPHARM (PHARMAMED),1111,PHARMAMED,8011117,HEXPHARM - AP 3
8,COXZB,OXYZONE 15 GR CREAM,801_1111,HEXPHARM (PHARMAMED),1111,PHARMAMED,8011117,HEXPHARM - AP 3
9,CPB59,PARAFFIN SECTION - ROUND BATH \t\t\t\t\t\t\t\t...,702_1111,EMP-HOSLAB (PHARMAMED),1111,PHARMAMED,7020002,EMP HOSLAB EMERGING


In [3]:
len(df)

22320

In [4]:
df.isnull().sum()

ITEM_CODE               0
ITEM_DESC               0
PRINCIPAL_CODE          0
PRINCIPAL_DESC          1
DIRECTORAT_ITEM_CODE    0
DIRECTORAT_ITEM_DESC    1
CLASS_PRODUCT_CODE      0
CLASS_PRODUCT_DESC      0
dtype: int64

In [5]:
df[df.isnull().any(axis=1)]

,ITEM_CODE,ITEM_DESC,PRINCIPAL_CODE,PRINCIPAL_DESC,DIRECTORAT_ITEM_CODE,DIRECTORAT_ITEM_DESC,CLASS_PRODUCT_CODE,CLASS_PRODUCT_DESC
20758,EPM_NA_0031208_003,EPM_NA_0031208_003,003_1172,None,1172,None,0031208,EPM_NA_0031208_003


In [6]:
df.head()

,ITEM_CODE,ITEM_DESC,PRINCIPAL_CODE,PRINCIPAL_DESC,DIRECTORAT_ITEM_CODE,DIRECTORAT_ITEM_DESC,CLASS_PRODUCT_CODE,CLASS_PRODUCT_DESC
0,00351,C-A-MATE XM 220 SYSTEM,702_1111,EMP-HOSLAB (PHARMAMED),1111,PHARMAMED,7024090,EMP-HOSLAB BLOOD ACCESORIES
1,CMYCD,MYCORAL CREAM 5 GR,803_1111,KALBE NIMITZ (PHARMAMED),1111,PHARMAMED,8011105,NIMITZ
2,CMYCF,MYCORAL CREAM 15 GR,803_1111,KALBE NIMITZ (PHARMAMED),1111,PHARMAMED,8011105,NIMITZ
3,01501,"DUODENAL LEVIN, CH 16",701_1111,EMP-HOSPEC (PHARMAMED),1111,PHARMAMED,7010890,EMP - HOSPEC LAIN-LAIN
4,CNZDA,NAZODERM CREAM 10 GR,002_1111,PT.DANKOS FARMA (PHARMAMED),1111,PHARMAMED,0021101,DANKOS GABUNGAN


In [7]:
df['metadata'] = df.apply(lambda row: ' '.join([str(row['ITEM_DESC']), str(row['PRINCIPAL_DESC']), str(row['DIRECTORAT_ITEM_DESC']), str(row['CLASS_PRODUCT_DESC'])]), axis=1)

In [8]:
df = df.drop(columns=['PRINCIPAL_CODE', 'DIRECTORAT_ITEM_CODE', 'CLASS_PRODUCT_CODE'])

In [9]:
df.head()

,ITEM_CODE,ITEM_DESC,PRINCIPAL_DESC,DIRECTORAT_ITEM_DESC,CLASS_PRODUCT_DESC,metadata
0,00351,C-A-MATE XM 220 SYSTEM,EMP-HOSLAB (PHARMAMED),PHARMAMED,EMP-HOSLAB BLOOD ACCESORIES,C-A-MATE XM 220 SYSTEM EMP-HOSLAB (PHARMAMED) ...
1,CMYCD,MYCORAL CREAM 5 GR,KALBE NIMITZ (PHARMAMED),PHARMAMED,NIMITZ,MYCORAL CREAM 5 GR KALBE NIMITZ (PHARMAMED) P...
2,CMYCF,MYCORAL CREAM 15 GR,KALBE NIMITZ (PHARMAMED),PHARMAMED,NIMITZ,MYCORAL CREAM 15 GR KALBE NIMITZ (PHARMAMED) P...
3,01501,"DUODENAL LEVIN, CH 16",EMP-HOSPEC (PHARMAMED),PHARMAMED,EMP - HOSPEC LAIN-LAIN,"DUODENAL LEVIN, CH 16 EMP-HOSPEC (PHARMAMED) P..."
4,CNZDA,NAZODERM CREAM 10 GR,PT.DANKOS FARMA (PHARMAMED),PHARMAMED,DANKOS GABUNGAN,NAZODERM CREAM 10 GR PT.DANKOS FARMA (PHARMAME...


In [10]:
# Specify the path to your text file
file_path = 'data/stopwords-id.txt'

# Read the file and store its contents as a list
with open(file_path, 'r') as file:
    lines = file.readlines()

# Remove trailing newline characters and any leading/trailing whitespace from each line
lines_stopwords = [line.rstrip('\n').strip() for line in lines]

In [11]:
encoder = TfidfVectorizer(stop_words=lines_stopwords, tokenizer=word_tokenize)

In [12]:
encoded_data = encoder.fit_transform(df.metadata)

/home/adhi/epm-recsys-dev/.env-epm/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [32]:
item_code = "MBAOA"

In [33]:
selected_metadata = df.loc[df['ITEM_CODE'] == item_code, 'metadata']
metadata_string = ' '.join(selected_metadata.values)
metadata_string

'MILNA BABY BISKUIT APEL JERUK 65G PT. SANGHIANG PERKASA (KND) KND MILNA BISC BAYI'

In [34]:
encoded_metadata = encoder.transform([metadata_string])

In [35]:
from sklearn.metrics.pairwise import cosine_distances

In [36]:
dist = cosine_distances(encoded_metadata, encoded_data)
dist

array([[0.9957887 , 0.9962364 , 0.99629837, ..., 0.98024478, 0.97867803,
        0.9805755 ]])

In [39]:
rec_idx = dist.argsort()[0, 1:11]
rec_idx

array([  780,   785,   791,   778,   768, 15874,   754,   764, 17061,
         763])

In [40]:
df.loc[rec_idx]

,ITEM_CODE,ITEM_DESC,PRINCIPAL_DESC,DIRECTORAT_ITEM_DESC,CLASS_PRODUCT_DESC,metadata
780,MBORB,MILNA BABY BISKUIT JERUK 130G,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,MILNA BABY BISKUIT JERUK 130G PT. SANGHIANG PE...
785,MBPIB,MILNA BABY BISKUIT PISANG 130G,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,MILNA BABY BISKUIT PISANG 130G PT. SANGHIANG P...
791,MBRGB,MILNA BABY BISKUIT ORIGINAL 130 G,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,MILNA BABY BISKUIT ORIGINAL 130 G PT. SANGHIAN...
778,MBKUB,MILNA BABY BISKUIT K HIJAU 130G,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,MILNA BABY BISKUIT K HIJAU 130G PT. SANGHIANG ...
768,MBBHB,MILNA BABY BISKUIT BERAS MERAH 130G,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,MILNA BABY BISKUIT BERAS MERAH 130G PT. SANGHI...
15874,MBRSC,MILNA BABY BISKUIT ORIGINAL 22G,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC SACHET,MILNA BABY BISKUIT ORIGINAL 22G PT. SANGHIANG ...
754,MAPLA,MIL BISCUIT APEL 75,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,MIL BISCUIT APEL 75 PT. SANGHIANG PERKASA (KN...
764,MBAPB,M.Biscuit Apel 130 Gr,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,M.Biscuit Apel 130 Gr PT. SANGHIANG PERKASA (K...
17061,MBAPC,M.BISCUIT AA+DHA APEL 300,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,M.BISCUIT AA+DHA APEL 300 PT. SANGHIANG PERKAS...
763,MBAPA,M.BISCUIT AA+DHA APEL 75,PT. SANGHIANG PERKASA (KND),KND,MILNA BISC BAYI,M.BISCUIT AA+DHA APEL 75 PT. SANGHIANG PERKAS...
